In [1]:
import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords

from scipy.stats import itemfreq
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,HashingVectorizer
from sklearn.metrics import confusion_matrix
import pickle
# pd.options.mode.chained_assignment = Nonedata1.head(10)

In [6]:
data1 = pd.read_csv('text_emotion.csv',encoding = "ISO-8859-1")

In [7]:
data1.head(10)

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...
5,1956968477,worry,xxxPEACHESxxx,Re-pinging @ghostridah14: why didn't you go to...
6,1956968487,sadness,ShansBee,"I should be sleep, but im not! thinking about ..."
7,1956968636,worry,mcsleazy,Hmmm. http://www.djhero.com/ is down
8,1956969035,sadness,nic0lepaula,@charviray Charlene my love. I miss you
9,1956969172,sadness,Ingenue_Em,@kelcouch I'm sorry at least it's Friday?


In [8]:
data1.shape


(40000, 4)

In [9]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   40000 non-null  int64 
 1   sentiment  40000 non-null  object
 2   author     40000 non-null  object
 3   content    40000 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.2+ MB


In [10]:
data1=data1[['tweet_id','sentiment','content']].copy()

In [11]:
data1.sentiment.value_counts()

neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: sentiment, dtype: int64

In [12]:
data1.sentiment = np.where((data1.sentiment == 'neutral') |(data1.sentiment == 'empty')|(data1.sentiment == 'boredom'),'neutral',data1.sentiment)


In [13]:
data1.sentiment = np.where((data1.sentiment == 'fun') |(data1.sentiment == 'enthusiasm'),'fun',data1.sentiment)

In [14]:
data1=data1[data1.sentiment !='neutral']

In [15]:
data1.sentiment.value_counts()

worry        8459
happiness    5209
sadness      5165
love         3842
fun          2535
surprise     2187
relief       1526
hate         1323
anger         110
Name: sentiment, dtype: int64

In [16]:
data2=pd.read_csv('tweets_clean.txt',sep='	',header=None)

In [17]:
data2.head(10)

,0,1,2
0,145353048817012736:,Thinks that @melbahughes had a great 50th birt...,:: surprise
1,144279638024257536:,"Como una expresión tan simple, una sola oració...",:: sadness
2,140499585285111809:,the moment when you get another follower and y...,:: joy
3,145207578270507009:,Be the greatest dancer of your life! practice ...,:: joy
4,139502146390470656:,eww.. my moms starting to make her annual rum ...,:: disgust
5,146042696899887106:,If ur heart hurts all the time for tht person ...,:: joy
6,145492569609084928:,"I feel awful, and it's way too freaking early....",:: joy
7,145903955229151232:,So chuffed for safc fans! Bet me dar comes in ...,:: joy
8,142717613234069504:,Making art and viewing art are different at th...,:: fear
9,144183822873927680:,"Soooo dooowwwn!! Move on, get some sleep... Me...",:: anger


In [18]:
data2.columns=['tweet_id','content','sentiment']

In [19]:
data2.sentiment = data2.sentiment.str.replace(':: ','')

In [20]:
data2.sentiment.value_counts()


joy         8240
surprise    3849
sadness     3830
fear        2816
anger       1555
disgust      761
Name: sentiment, dtype: int64

In [21]:
data = data1.append(data2)

C:\Users\Saransh\AppData\Local\Temp\ipykernel_24168\2571438737.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data1.append(data2)


In [22]:
data.head(10)

,tweet_id,sentiment,content
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,fun,wants to hang out with friends SOON!
5,1956968477,worry,Re-pinging @ghostridah14: why didn't you go to...
6,1956968487,sadness,"I should be sleep, but im not! thinking about ..."
7,1956968636,worry,Hmmm. http://www.djhero.com/ is down
8,1956969035,sadness,@charviray Charlene my love. I miss you
9,1956969172,sadness,@kelcouch I'm sorry at least it's Friday?
11,1956969531,worry,Choked on her retainers
12,1956970047,sadness,Ugh! I have to beat this stupid song to get to...


In [23]:
# data.sentiment = np.where((data.sentiment == 'disgust') |(data.sentiment == 'hate'),'hate',data.sentiment)

In [24]:
data.sentiment.value_counts()

sadness      8995
worry        8459
joy          8240
surprise     6036
happiness    5209
love         3842
fear         2816
fun          2535
anger        1665
relief       1526
hate         1323
disgust       761
Name: sentiment, dtype: int64

In [25]:
data=data[data.sentiment.isin(['sadness','anger','happiness','fear','love'])]

In [26]:
data.sentiment.value_counts()

sadness      8995
happiness    5209
love         3842
fear         2816
anger        1665
Name: sentiment, dtype: int64

In [27]:
data['content']=data['content'].str.replace('[^A-Za-z0-9\s]+', '')

C:\Users\Saransh\AppData\Local\Temp\ipykernel_24168\2657433801.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['content']=data['content'].str.replace('[^A-Za-z0-9\s]+', '')


In [28]:
data['content']=data['content'].str.replace('http\S+|www.\S+', '', case=False)

C:\Users\Saransh\AppData\Local\Temp\ipykernel_24168\485729106.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['content']=data['content'].str.replace('http\S+|www.\S+', '', case=False)


In [29]:
data['content']=data['content'].str.lower()

In [30]:
target=data.sentiment
data = data.drop(['sentiment'],axis=1)

# target.tolist()

In [31]:
le=LabelEncoder()
target=le.fit_transform(target)

# target.tolist()

In [32]:
X_train, X_test, y_train, y_test = train_test_split(data,target,stratify=target,test_size=0.4, random_state=42)

In [33]:
itemfreq(y_train)

C:\Users\Saransh\AppData\Local\Temp\ipykernel_24168\3655988563.py:1: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  itemfreq(y_train)


array([[   0,  999],
       [   1, 1690],
       [   2, 3125],
       [   3, 2305],
       [   4, 5397]], dtype=int64)

In [34]:
itemfreq(y_test)

C:\Users\Saransh\AppData\Local\Temp\ipykernel_24168\4074339663.py:1: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  itemfreq(y_test)


array([[   0,  666],
       [   1, 1126],
       [   2, 2084],
       [   3, 1537],
       [   4, 3598]], dtype=int64)

In [35]:
X_train.content

19776                        so glad sam is in a good mood
32338    jonathanrknight thank u sweetie cant wait to s...
32505    wishing mom happy mothers day and ofcourse hap...
32570    i had a great night with justine and the boys ...
2386     and faith have equivalent power the first move...
                               ...                        
19844    now for an exciting night of paper writing gra...
17413    i hate having good dreams and then waking up k...
13298    tommcfly wishing i could be in so paulo for th...
18817    waiting for the denver game to come on but i d...
22027    joolzgirl oh no u must get 1lol mum still isnt...
Name: content, Length: 13516, dtype: object

In [59]:
# Extracting features from text files
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train.content)
X_test_counts =count_vect.transform(X_test.content)
print('Shape of Term Frequency Matrix: ',X_train_counts.shape)

pickle.dump(count_vect, open("vector.pickle", 'wb'))

Shape of Term Frequency Matrix:  (13516, 24881)


In [38]:
import pickle
pickle.dump(X_test.content, open("x_test_content.pickle", 'wb'))

In [39]:
pickle.dump(X_train.content, open("x_train_content.pickle", 'wb'))

In [40]:
# Machine Learning
# Training Naive Bayes (NB) classifier on training data.
clf = MultinomialNB().fit(X_train_counts,y_train)
predicted = clf.predict(X_test_counts)
nb_clf_accuracy = np.mean(predicted == y_test) * 100
print(nb_clf_accuracy)

57.56297858173344


In [36]:
predicted

array([4, 2, 4, ..., 4, 2, 4])

In [37]:
def print_acc(model):
    predicted = model.predict(X_test.content)
    accuracy = np.mean(predicted == y_test) * 100
    print(accuracy)

In [38]:
nb_clf = Pipeline([('vect', CountVectorizer()), ('clf', MultinomialNB())])
nb_clf = nb_clf.fit(X_train.content,y_train)
print_acc(nb_clf)

57.56297858173344


In [39]:
nb_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
nb_clf = nb_clf.fit(X_train.content,y_train)
print_acc(nb_clf)

48.707135723005216


In [40]:
nb_clf = Pipeline([('vect', HashingVectorizer(n_features=2500,alternate_sign=False)), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
nb_clf = nb_clf.fit(X_train.content,y_train)
print_acc(nb_clf)

51.8144490067695


In [41]:
confusion_matrix(y_test,predicted)

array([[  61,   35,   17,    5,  548],
       [   6,  474,   29,    6,  611],
       [   0,    9,  859,  163, 1053],
       [   1,    2,  389,  420,  725],
       [   7,   57,  134,   27, 3373]], dtype=int64)

In [42]:
stop_words = set(stopwords.words('english'))
nb_clf = Pipeline([('vect', CountVectorizer(stop_words=stop_words)), ('clf', MultinomialNB())])
nb_clf = nb_clf.fit(X_train.content,y_train)
print_acc(nb_clf)

58.461879924536674


In [43]:
nb_clf = Pipeline([('vect', CountVectorizer(stop_words=stop_words)), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
nb_clf = nb_clf.fit(X_train.content,y_train)
print_acc(nb_clf)

51.370547109088896


In [44]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)])
X_train.loc[:,'content'] = X_train['content'].apply(lemmatize_text)
X_test.loc[:,'content'] = X_test['content'].apply(lemmatize_text)

C:\Users\Saransh\AppData\Local\Temp\ipykernel_21300\655015578.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[:,'content'] = X_train['content'].apply(lemmatize_text)
C:\Users\Saransh\AppData\Local\Temp\ipykernel_21300\655015578.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.loc[:,'content'] = X_test['content'].apply(lemmatize_text)


In [45]:
nb_clf = Pipeline([('vect', CountVectorizer(stop_words=stop_words)), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
nb_clf = nb_clf.fit(X_train.content,y_train)
print_acc(nb_clf)

50.971035401176344


In [41]:
import pickle

pickle.dump(clf, open("model.pickle", 'wb'))

In [2]:
loaded_model = pickle.load(open("model.pickle", 'rb'))

In [43]:
y_pred = loaded_model.predict(X_test_counts)

In [44]:
y_pred

array([4, 2, 4, ..., 4, 2, 4])

In [45]:
y_pred = clf.predict(X_test_counts)

In [46]:
x_test_content = pickle.load(open("x_test_content.pickle", 'rb'))

In [47]:
x_train_content = pickle.load(open("x_train_content.pickle", 'rb'))

In [3]:
vector = pickle.load(open("vector.pickle", 'rb'))

In [4]:
text_vector = vector.transform(["Lol"])

In [5]:
loaded_model.predict(text_vector)


array([2])

In [79]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(x_train_content)
X_test_counts =count_vect.transform(x_test_content)
y = loaded_model.predict(X_test_counts)

ValueError: dimension mismatch

In [53]:
X_train.content

19776                        so glad sam is in a good mood
32338    jonathanrknight thank u sweetie cant wait to s...
32505    wishing mom happy mother day and ofcourse happ...
32570    i had a great night with justine and the boy a...
2386     and faith have equivalent power the first move...
                               ...                        
19844    now for an exciting night of paper writing gra...
17413    i hate having good dream and then waking up kn...
13298    tommcfly wishing i could be in so paulo for th...
18817    waiting for the denver game to come on but i d...
22027    joolzgirl oh no u must get 1lol mum still isnt...
Name: content, Length: 13516, dtype: object

In [86]:
count_vect = CountVectorizer()
X_train_counts = count_vect.transform(X_train.content)

loaded_model.predict(X_train_counts)

ValueError: dimension mismatch